# Model Monitoring

In [1]:
# Import libraries
from datetime import datetime, timedelta, timezone
import json
import os
import re
import boto3
from threading import Thread
import pandas as pd
from sagemaker import get_execution_role, session, Session, image_uris
from sagemaker.s3 import S3Downloader, S3Uploader
from sagemaker.processing import ProcessingJob
from sagemaker.serializers import CSVSerializer
from sagemaker.model import Model
from sagemaker.model_monitor import DataCaptureConfig
from sagemaker.predictor import Predictor
from time import gmtime, strftime, sleep
from sagemaker.model_monitor import ModelQualityMonitor
from sagemaker.model_monitor import EndpointInput
from sagemaker.model_monitor.dataset_format import DatasetFormat
from sagemaker.model_monitor import CronExpressionGenerator
from tqdm.notebook import tqdm
from sagemaker.model_monitor import DefaultModelMonitor

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


In [2]:
# Create CloudWatch client
cw_client = boto3.Session().client("cloudwatch")
namespace = "aws/sagemaker/Endpoints/model-metrics"

In [3]:
# Setup boto and sagemaker session
sagemaker_session = Session()
role = get_execution_role()
region = sagemaker_session.boto_region_name

# Setup S3 bucket
bucket = sagemaker_session.default_bucket()
print("Bucket:", bucket)
prefix = f"sagemaker/FoodLens-ModelQualityMonitor-{datetime.now():%Y-%m-%d-%H-%M-%S}"

# S3 prefixes
data_capture_prefix = f"{prefix}/datacapture"
s3_capture_upload_path = f"s3://{bucket}/{data_capture_prefix}"

ground_truth_upload_path = (
    f"s3://{bucket}/{prefix}/ground_truth_data/{datetime.now():%Y-%m-%d-%H-%M-%S}"
)

reports_prefix = f"{prefix}/reports"
s3_report_path = f"s3://{bucket}/{reports_prefix}"

print(f"Capture path: {s3_capture_upload_path}")
print(f"Ground truth path: {ground_truth_upload_path}")
print(f"Report path: {s3_report_path}")

Bucket: sagemaker-us-east-1-381492037991
Capture path: s3://sagemaker-us-east-1-381492037991/sagemaker/FoodLens-ModelQualityMonitor-2025-10-15-23-59-25/datacapture
Ground truth path: s3://sagemaker-us-east-1-381492037991/sagemaker/FoodLens-ModelQualityMonitor-2025-10-15-23-59-25/ground_truth_data/2025-10-15-23-59-25
Report path: s3://sagemaker-us-east-1-381492037991/sagemaker/FoodLens-ModelQualityMonitor-2025-10-15-23-59-25/reports


In [4]:
import boto3
print(boto3.Session().region_name)
print(boto3.client("sts").get_caller_identity())


us-east-1
{'UserId': 'AROAVRUVSRFTVVZM55U34:SageMaker', 'Account': '381492037991', 'Arn': 'arn:aws:sts::381492037991:assumed-role/LabRole/SageMaker', 'ResponseMetadata': {'RequestId': '2b297211-b372-4e2e-9071-a54207933f6c', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '2b297211-b372-4e2e-9071-a54207933f6c', 'x-amz-sts-extended-request-id': 'MTp1cy1lYXN0LTE6MTc2MDU3Mjc2NjIyNjpHOlAzR05QbTNp', 'content-type': 'text/xml', 'content-length': '432', 'date': 'Wed, 15 Oct 2025 23:59:26 GMT'}, 'RetryAttempts': 0}}


In [5]:
import boto3
sm = boto3.client("sagemaker", region_name="us-east-1")

for job in sm.list_training_jobs(NameContains="xgb-nutriscore")["TrainingJobSummaries"]:
    print(job["TrainingJobName"])

## Deploy Pre-Trained Model to Live Endpoint

In [15]:
# Initialize the sagemaker client
sagemaker_client = boto3.client("sagemaker")

# Specify model
image_uri = image_uris.retrieve(framework="xgboost", region=region, version="1.7-1")
instance_type = 'ml.m5.xlarge'
model_name = 'nutrition-score-xgb-2025-10-15-05-02-13' # get from notebook 04
response = sagemaker_client.describe_model(ModelName=model_name)
model_url = response['PrimaryContainer']['ModelDataUrl']
model = Model(image_uri=image_uri, model_data=model_url, role=role, sagemaker_session=sagemaker_session)

In [16]:
endpoint_name = f"xgb-nutriscore-model-quality-monitor-{datetime.now():%Y-%m-%d-%H-%M-%S}"
print("EndpointName: ", endpoint_name)

# Enable data capture
data_capture_config = DataCaptureConfig(
    enable_capture=True, sampling_percentage=100, destination_s3_uri=s3_capture_upload_path
)

# Deploy the model and wait for it to be in service
print("Deploying endpoint....")
model.deploy(
    initial_instance_count=1,
    instance_type=instance_type,
    endpoint_name=endpoint_name,
    data_capture_config=data_capture_config,
)

print(f"\nEndpoint '{endpoint_name}' in Service.")

EndpointName:  xgb-nutriscore-model-quality-monitor-2025-10-15-07-33-09
Deploying endpoint....
------!
Endpoint 'xgb-nutriscore-model-quality-monitor-2025-10-15-07-33-09' in Service.


In [17]:
# Create predictor object
predictor = Predictor(
    endpoint_name=endpoint_name, 
    sagemaker_session=sagemaker_session, 
    serializer=CSVSerializer()
)

## Setup Infrastructure Monitoring

In [18]:
# Create a CloudWatch alarm for model latency
alarm_name = "NUTRISCORE_MODEL_LATENCY_HIGH"
alarm_desc = "Trigger an alarm when the average model latency exceeds 200ms."
infrastructure_metric_name = 'ModelLatency'
cw_latency_dimensions = [
    {"Name": "Endpoint", "Value": endpoint_name},
    {"Name": "MonitoringSchedule", "Value": 'AllTraffic'},
]

# Create the alarm
cw_client.put_metric_alarm(
## fill here
)

print(f"CloudWatch alarm '{alarm_name}' for infrastructure latency has been created.")

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:11                                                                                   │
│                                                                                                  │
│    8 ]                                                                                           │
│    9                                                                                             │
│   10 # Create the alarm                                                                          │
│ ❱ 11 cw_client.put_metric_alarm(                                                                 │
│   12 ## fill here                                                                                │
│   13 )                                                                                           │
│   14                                                                                             │
│                                                                                                  │
│ /opt/conda/lib/python3.12/site-packages/botocore/client.py:569 in _api_call                      │
│                                                                                                  │
│    566 │   │   │   │   │   f"{py_operation_name}() only accepts keyword arguments."              │
│    567 │   │   │   │   )                                                                         │
│    568 │   │   │   # The "self" in this scope is referring to the BaseClient.                    │
│ ❱  569 │   │   │   return self._make_api_call(operation_name, kwargs)                            │
│    570 │   │                                                                                     │
│    571 │   │   _api_call.__name__ = str(py_operation_name)                                       │
│    572                                                                                           │
│                                                                                                  │
│ /opt/conda/lib/python3.12/site-packages/botocore/client.py:980 in _make_api_call                 │
│                                                                                                  │
│    977 │   │   │   # Pass arbitrary endpoint info with the Request                               │
│    978 │   │   │   # for use during construction.                                                │
│    979 │   │   │   request_context['endpoint_properties'] = properties                           │
│ ❱  980 │   │   request_dict = self._convert_to_request_dict(                                     │
│    981 │   │   │   api_params=api_params,                                                        │
│    982 │   │   │   operation_model=operation_model,                                              │
│    983 │   │   │   endpoint_url=endpoint_url,                                                    │
│                                                                                                  │
│ /opt/conda/lib/python3.12/site-packages/botocore/client.py:1047 in _convert_to_request_dict      │
│                                                                                                  │
│   1044 │   │   headers=None,                                                                     │
│   1045 │   │   set_user_agent_header=True,                                                       │
│   1046 │   ):                                                                                    │
│ ❱ 1047 │   │   request_dict = self._serializer.serialize_to_request(                             │
│   1048 │   │   │   api_params, operation_model                                                   │
│   1049 │   │   )                                                                                 │
│   1050 │   │   if not self._client_config.inject_host_prefi

## Setup Data Quality Monitor

In [33]:
#Ignore this section, but use already cleaned csv train_scaled_ready_final.csv
#the data cleanup step removes empty and non-numeric columns and adds headers 
#This is make csv compatible with SageMaker’s data requirements.  

import io

# Define S3 keys (paths inside your bucket)
key = "nutriscore-prediction-xgboost/train/train_scaled_ready2.csv"
final_ready_key = "nutriscore-prediction-xgboost/train/train_scaled_ready_final.csv"


s3 = boto3.client("s3", region_name=region)

# Download, clean, and upload ---
obj = s3.get_object(Bucket=bucket, Key=key)
df = pd.read_csv(io.BytesIO(obj["Body"].read()))

# Drop empty columns and keep only numeric columns
df_clean = (
    df.dropna(axis=1, how="all")
      .select_dtypes(include=["number"])
)

print(f"Final shape after cleanup: {df_clean.shape}")
print(f"Columns: {list(df_clean.columns)}")

# Save locally and upload back to S3
local_path = "/tmp/train_scaled_ready_final.csv"
df_clean.to_csv(local_path, index=False)
s3.upload_file(local_path, bucket, final_ready_key)

print(f"Final ready CSV uploaded to s3://{bucket}/{final_ready_key}")

Final shape after cleanup: (39955, 23)
Columns: ['code', 'product_name', 'nova_group', 'additives_n', 'ingredients_n', 'nutriscore_score', 'energy_100g', 'salt_100g', 'carbohydrates_100g', 'cholesterol_100g', 'sodium_100g', 'fiber_100g', 'fruits_vegetables_legumes_estimate_from_ingredients_100g', 'sugars_100g', 'saturated_fat_100g', 'trans_fat_100g', 'fat_100g', 'proteins_100g', 'fruits_vegetables_nuts_estimate_from_ingredients_100g', 'energy_kcal_100g', 'nova_group_100g', 'nutrition_score_fr_100g', 'energy_kj_100g']
Final ready CSV uploaded to s3://sagemaker-us-east-1-381492037991/nutriscore-prediction-xgboost/train/train_scaled_ready_final.csv


In [23]:
from sagemaker.model_monitor import DefaultModelMonitor, DatasetFormat
from datetime import datetime


# Build baseline from scaled training data. Use the updated csv file. 
train_s3_path = f's3://{bucket}/nutriscore-prediction-xgboost/train/train_scaled_ready_final.csv'

# S3 path for data quality reports
data_quality_report_path = f"s3://{bucket}/nutriscore-prediction-xgboost/data-quality-reports"

# Create Data Quality Monitor
data_quality_monitor = DefaultModelMonitor(
    role=role,
    instance_count=1,
    instance_type='ml.m5.xlarge',
    volume_size_in_gb=20,
    max_runtime_in_seconds=3600,
    base_job_name='nutriscore-data-quality-baseline',
    sagemaker_session=sagemaker_session
)

data_quality_baseline_job_name = f"nutriscore-data-quality-baseline-job-{datetime.now():%Y-%m-%d-%H-%M-%S}"

print("Starting Data Quality baseline suggestion job...")

# Run baseline job
data_quality_monitor.suggest_baseline(
    baseline_dataset=train_s3_path,
    dataset_format=DatasetFormat.csv(header=True),
    output_s3_uri=data_quality_report_path,
    wait=True,
    job_name=data_quality_baseline_job_name
)

print("\nData Quality baseline job complete.")


INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.
INFO:sagemaker:Creating processing-job with name nutriscore-data-quality-baseline-job-2025-10-16-01-00-22


Starting Data Quality baseline suggestion job...
.................2025-10-16 01:03:10.458083: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2025-10-16 01:03:10.458112: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2025-10-16 01:03:12.034444: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2025-10-16 01:03:12.034476: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2025-10-16 01:03:12.034497: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (ip-10-2-198-177.ec2.internal): /proc/driver/nvidia/versi

In [18]:
data_quality_monitor.latest_baselining_job.describe()


{'ProcessingInputs': [{'InputName': 'baseline_dataset_input',
   'AppManaged': False,
   'S3Input': {'S3Uri': 's3://sagemaker-us-east-1-381492037991/nutriscore-prediction-xgboost/train/train_scaled_ready_final.csv',
    'LocalPath': '/opt/ml/processing/input/baseline_dataset_input',
    'S3DataType': 'S3Prefix',
    'S3InputMode': 'File',
    'S3DataDistributionType': 'FullyReplicated',
    'S3CompressionType': 'None'}}],
 'ProcessingOutputConfig': {'Outputs': [{'OutputName': 'monitoring_output',
    'S3Output': {'S3Uri': 's3://sagemaker-us-east-1-381492037991/nutriscore-prediction-xgboost/data-quality-reports',
     'LocalPath': '/opt/ml/processing/output',
     'S3UploadMode': 'EndOfJob'},
    'AppManaged': False}]},
 'ProcessingJobName': 'nutriscore-data-quality-baseline-job-2025-10-15-20-53-27',
 'ProcessingResources': {'ClusterConfig': {'InstanceCount': 1,
   'InstanceType': 'ml.m5.xlarge',
   'VolumeSizeInGB': 20}},
 'StoppingCondition': {'MaxRuntimeInSeconds': 3600},
 'AppSpecif

In [16]:
from datetime import datetime

data_quality_schedule_name = f"nutriscore-data-quality-schedule-{datetime.now():%Y-%m-%d-%H-%M-%S}"

statistics_file = (
    "s3://sagemaker-us-east-1-381492037991/"
    "nutriscore-prediction-xgboost/data-quality-reports/statistics.json" # You can copy this path from your s3. 
)
constraints_file = (
    "s3://sagemaker-us-east-1-381492037991/"
    "nutriscore-prediction-xgboost/data-quality-reports/constraints.json"
)

print(f"Creating Data Quality monitoring schedule: {data_quality_schedule_name}")

data_quality_monitor.create_monitoring_schedule(
    monitor_schedule_name=data_quality_schedule_name,
    endpoint_input="xgb-nutriscore-model-quality-monitor-2025-10-15-07-33-09",  # Replace with your endpoint name from the code above 
    output_s3_uri=data_quality_report_path,
    statistics=statistics_file,
    constraints=constraints_file,
    schedule_cron_expression="cron(0 * ? * * *)"  #  Runs every hour
)

print(" Data Quality monitoring schedule created successfully.")


Creating Data Quality monitoring schedule: nutriscore-data-quality-schedule-2025-10-16-00-21-39


INFO:sagemaker.model_monitor.model_monitoring:Creating Monitoring Schedule with name: nutriscore-data-quality-schedule-2025-10-16-00-21-39


✅ Data Quality monitoring schedule created successfully.


## Generate Baseline for Model Quality Performance

In [ ]:
# Get validation dataset
# get from previous notebook
val_s3_path = 's3://sagemaker-us-east-1-654654380268/nutriscore-prediction-xgboost/validation/val_scaled.csv' 
!aws s3 cp {val_s3_path} ./
val_local_path = './val_scaled.csv'

In [ ]:
# Your validation dataset should h first column
limit = 500 # number of samples for baseline
baseline_file_name = 'val_pred_baseline.csv'
i = 0

# Create a new file for your baseline data
with open(f"{baseline_file_name}", "w") as baseline_file:
    # Header for a regression baseline
    baseline_file.write("prediction,label\n")
    
    # Open validation data file
    with open(val_local_path, "r") as f:
        for row in f:
            # With true score in first column
            (label, input_cols) = row.split(",", 1)
            
            # Get the predicted score from the endpoint
            predicted_score = float(predictor.predict(input_cols))
            
            # Write the predicted score and the true label to the baseline file
            baseline_file.write(f"{predicted_score},{label.strip()}\n")
            
            i += 1
            if i >= limit:
                break
            print(".", end="", flush=True)
            sleep(0.5)
print()
print("Done!")

In [ ]:
# Examine predictions from model
!head {baseline_file_name}

In [ ]:
# Upload predictions as baseline dataset
baseline_prefix = prefix + "/baselining"
baseline_data_prefix = baseline_prefix + "/data"
baseline_results_prefix = baseline_prefix + "/results"

baseline_data_uri = f"s3://{bucket}/{baseline_data_prefix}"
baseline_results_uri = f"s3://{bucket}/{baseline_results_prefix}"
print(f"Baseline data uri: {baseline_data_uri}")
print(f"Baseline results uri: {baseline_results_uri}")

In [ ]:
# Upload baseline dataset
baseline_dataset_uri = S3Uploader.upload(f"{baseline_file_name}", baseline_data_uri)
baseline_dataset_uri

In [ ]:
# Create the model quality monitoring object
nutriscore_model_quality_monitor = ModelQualityMonitor(
    role=role,
    instance_count=1,
    instance_type=instance_type,
    volume_size_in_gb=20,
    max_runtime_in_seconds=1800,
    sagemaker_session=sagemaker_session,
)

In [ ]:
# Name of the model quality baseline job
baseline_job_name = f"xgb-nutriscore-model-baseline-job-{datetime.now():%Y-%m-%d-%H%M}"

In [ ]:
# Execute the baseline suggestion job
job = nutriscore_model_quality_monitor.suggest_baseline(
    job_name=baseline_job_name,
    baseline_dataset=baseline_dataset_uri,
    dataset_format=DatasetFormat.csv(header=False),
    output_s3_uri=baseline_results_uri,
    problem_type="Regression",
    inference_attribute="prediction", # model output
    ground_truth_attribute="label", # true score
)
job.wait(logs=False)

## Explore Results of Baseline Job

In [ ]:
baseline_job = nutriscore_model_quality_monitor.latest_baselining_job

In [ ]:
# View metrics
binary_metrics = baseline_job.baseline_statistics().body_dict["regression_metrics"]
pd.json_normalize(binary_metrics).T

In [ ]:
# View constraints
regression_constraints = pd.DataFrame(baseline_job.suggested_constraints().body_dict["regression_constraints"]).T
regression_constraints

## Setup Continuous Model Monitoring

In [ ]:
# Get some samples from scaled production data split
# From notebook 04
prod_scaled_path = 's3://sagemaker-us-east-1-654654380268/nutriscore-prediction-xgboost/prod/prod_scaled.csv'
prod_scaled_df = pd.read_csv(prod_scaled_path)

In [ ]:
# Take a sample for the test run (500 random rows)
sample_traffic_df = prod_scaled_df.sample(n=500)
print(f"Using a sample of {len(sample_traffic_df)} rows from the production dataset.")

In [ ]:
# Generate prediction data on sample data

# Store ground truth with inference ids
ground_truth_labels_with_ids = []

# First column is the label (true score) and the rest are features
label_column = sample_traffic_df.columns[0]
feature_columns = sample_traffic_df.columns[1:]

print("Sending pre-scaled production samples for inference...")

# Send samples for inference
for index, row in tqdm(sample_traffic_df.iterrows(), total=sample_traffic_df.shape[0]):
    features_payload = ",".join(map(str, row[feature_columns].values))
    
    # Invoke the endpoint
    sagemaker_session.sagemaker_runtime_client.invoke_endpoint(
        EndpointName=endpoint_name,
        ContentType="text/csv",
        Body=features_payload,
        InferenceId=str(index), # use index as unique id
    )
    
    # Store the true label and its corresponding ID for the ground truth upload
    ground_truth_labels_with_ids.append({
        "inference_id": index,
        "label": row[label_column]
    })
    sleep(0.2)

In [ ]:
# View captured data
print("Waiting for captures to show up", end="")
for _ in range(120):
    capture_files = sorted(S3Downloader.list(f"{s3_capture_upload_path}/{endpoint_name}"))
    if capture_files:
        capture_file = S3Downloader.read_file(capture_files[-1]).split("\n")
        capture_record = json.loads(capture_file[0])
        if "inferenceId" in capture_record["eventMetadata"]:
            break
    print(".", end="", flush=True)
    sleep(1)
print()
print("Found Capture Files:")
print("\n ".join(capture_files[-3:]))

In [ ]:
# View single capture
print(json.dumps(capture_record, indent=2))

In [ ]:
# Format and upload the true labels
ground_truth_records = []
for item in ground_truth_labels_with_ids:
    record = {
        "groundTruthData": {
            "data": str(item['label']), # The true nutrition score
            "encoding": "CSV",
        },
        "eventMetadata": {
            "eventId": item['inference_id'],
        },
        "eventVersion": "0",
    }
    ground_truth_records.append(json.dumps(record))

# Convert the list of JSON strings into a single string with newlines
ground_truth_data_to_upload = "\n".join(ground_truth_records)

# Upload to the S3 path the monitor is watching
target_s3_uri = f"{ground_truth_upload_path}/{datetime.utcnow():%Y/%m/%d/%H}/ground_truth.jsonl"

print(f"\nUploading {len(ground_truth_records)} ground truth records to {target_s3_uri}")
S3Uploader.upload_string_as_file_body(ground_truth_data_to_upload, target_s3_uri)

## Create Monitoring Schedule

In [ ]:
# Set monitor schedule name
nutriscore_monitor_schedule_name = f"nutriscore-monitoring-schedule-{datetime.now():%Y-%m-%d-%H-%M-%S}"

In [ ]:
# EndpointInput for regression
endpointInput = EndpointInput(
    endpoint_name=predictor.endpoint_name,
    inference_attribute="0", # first column contains inference
    destination="/opt/ml/processing/input_data",
)

In [ ]:
# Create the monitoring schedule to execute every hour
response = nutriscore_model_quality_monitor.create_monitoring_schedule(
    monitor_schedule_name=nutriscore_monitor_schedule_name,
    endpoint_input=endpointInput,
    output_s3_uri=baseline_results_uri,
    problem_type="Regression",
    ground_truth_input=ground_truth_upload_path,
    constraints=baseline_job.suggested_constraints(),
    schedule_cron_expression=CronExpressionGenerator.hourly(),
    enable_cloudwatch_metrics=True,
)

In [ ]:
# Examine schedule on monitor
nutriscore_model_quality_monitor.describe_schedule()

In [ ]:
# Initially there will be no executions since the first execution happens at the top of the hour
executions = nutriscore_model_quality_monitor.list_executions()
executions

In [ ]:
# Wait for the first execution of the monitoring_schedule
print("Waiting for first execution", end="")
while True:
    execution = nutriscore_model_quality_monitor.describe_schedule().get(
        "LastMonitoringExecutionSummary"
    )
    if execution:
        break
    print(".", end="", flush=True)
    sleep(10)
print()
print("Execution found!")

In [ ]:
# View execution details
while not executions:
    executions = nutriscore_model_quality_monitor.list_executions()
    print(".", end="", flush=True)
    sleep(10)
latest_execution = executions[-1]
latest_execution.describe()

In [ ]:
# View execution status
status = execution["MonitoringExecutionStatus"]

while status in ["Pending", "InProgress"]:
    print("Waiting for execution to finish", end="")
    latest_execution.wait(logs=False)
    latest_job = latest_execution.describe()
    print()
    print(f"{latest_job['ProcessingJobName']} job status:", latest_job["ProcessingJobStatus"])
    print(
        f"{latest_job['ProcessingJobName']} job exit message, if any:",
        latest_job.get("ExitMessage"),
    )
    print(
        f"{latest_job['ProcessingJobName']} job failure reason, if any:",
        latest_job.get("FailureReason"),
    )
    sleep(
        30
    )  # model quality executions consist of two Processing jobs, wait for second job to start
    latest_execution = nutriscore_model_quality_monitor.list_executions()[-1]
    execution = nutriscore_model_quality_monitor.describe_schedule()["LastMonitoringExecutionSummary"]
    status = execution["MonitoringExecutionStatus"]

print("Execution status is:", status)

if status != "Completed":
    print(execution)
    print(
        "====STOP==== \n No completed executions to inspect further. Please wait till an execution completes or investigate previously reported failures."
    )

In [ ]:
# View generated report uri
latest_execution = nutriscore_model_quality_monitor.list_executions()[-1]
report_uri = latest_execution.describe()["ProcessingOutputConfig"]["Outputs"][0]["S3Output"][
    "S3Uri"
]
print("Report Uri:", report_uri)

In [ ]:
# View violations generated by monitoring schedule
pd.options.display.max_colwidth = None
violations = latest_execution.constraint_violations().body_dict["violations"]
violations_df = pd.json_normalize(violations)
violations_df.head(10)

## Create Quality CloudWatch Alarm

In [ ]:
# Get thresholds from baseline constraints
rmse_threshold = regression_constraints.loc['rmse', 'threshold']
rmse_operator = regression_constraints.loc['rmse', 'comparison_operator']
print(f"RMSE Threshold from baseline constraints: {rmse_threshold}")
print(f"RMSE Comparison Operator from baseline constraints: {rmse_operator}")

In [ ]:
# Create a CloudWatch Alarm for a regression metrics
print("Creating CloudWatch alarm for RMSE...")
alarm_name = "NUTRISCORE_MODEL_RMSE_DRIFT"
alarm_desc = "Trigger an alarm when the model's RMSE exceeds the baseline threshold."
cw_quality_dimensions = [
    {"Name": "Endpoint", "Value": endpoint_name},
    {"Name": "MonitoringSchedule", "Value": nutriscore_monitor_schedule_name},
]

cw_client.put_metric_alarm(
    AlarmName=alarm_name,
    AlarmDescription=alarm_desc,
    ActionsEnabled=True,
    MetricName="rmse",
    Namespace=namespace,
    Statistic="Average",
    Dimensions=cw_quality_dimensions,
    Period=3600, # Checks every hour
    EvaluationPeriods=1,
    DatapointsToAlarm=1,
    Threshold=rmse_threshold,
    ComparisonOperator=rmse_operator,
    TreatMissingData="breaching",
)

## Cleanup

In [ ]:
# Delete monitoring schedules
nutriscore_model_quality_monitor.delete_monitoring_schedule()
data_quality_monitor.delete_monitoring_schedule()
predictor.delete_endpoint()